In [2]:
import pandas as pd
import pickle
import numpy as np
import math
warf = pd.read_pickle('data/cleaned_warfarin.pkl')
true_values = []
with open ('data/true_labels.pkl', 'rb') as f:
    true_values = pickle.load(f)

In [3]:
len(warf)

5528

In [4]:
len(list(warf))

94

In [5]:
train_len = math.ceil(len(true_values)*0.80)
test_len = len (true_values) - train_len

In [6]:
arms = ["low", "medium", "high"]
accuracy = []
regrets = []
#train
r_total = 0
for j in range(10):
    accuracy_list = []
    regret_list = []
    regret = 0
    risk = 0
    A_map = {}
    b_map = {} 
    for arm in arms:
        A_map[arm] = np.identity(len(list(warf))) 
        b_map[arm] = np.zeros(len(list(warf)))
    
    indices = np.random.permutation(list(range(len(warf))))
    for i in indices[:train_len]:
        feature = np.array(warf.iloc[i], dtype=float)
        
        # iterate through actions
        p = {}
        max_value = -np.inf
        max_arm = "low"
        for arm in arms:
            theta = np.matmul(np.linalg.inv(A_map[arm]), b_map[arm])
            p = np.matmul(theta.T, feature) + \
                np.sqrt(np.matmul(np.matmul(feature.T,np.linalg.inv(A_map[arm])), feature))
            
            if p == max_value:
                if np.random.choice(a=[False, True]): max_arm = arm
            elif p > max_value:
                max_value = p
                max_arm = arm
        
        # observe reward and update A and b
        r = 0 if max_arm == true_values[i] else -1
        regret -= r
        A_map[max_arm] += np.outer(feature, feature) 
        b_map[max_arm] += r*feature
        accuracy_list.append((i+1 - regret)/(i+1))
        regret_list.append(regret)

    correct_count = 0
    count = 0
    for i in indices[train_len:]:
        count += 1
        feature = np.array(warf.iloc[i], dtype=float)
        # iterate through actions
        p = {}
        max_value = -np.inf
        max_arm = "low"
        for arm in arms:
            theta = np.matmul(np.linalg.inv(A_map[arm]), b_map[arm])
            p = np.matmul(theta.T, feature) + \
                    np.sqrt(np.matmul(np.matmul(feature.T,np.linalg.inv(A_map[arm])), feature))
            if p == max_value:
                if np.random.choice(a=[False, True]): max_arm = arm
            elif p > max_value:
                max_value = p
                max_arm = arm
        # observe reward and update A and b
        if max_arm == true_values[i]: correct_count += 1
        if (max_arm == 'high' and true_values[i] == 'low') or (max_arm == 'low' and true_values[i] == 'high'):
            risk += 1
    print(risk/count)
    print(correct_count/count)
    accuracy.append(correct_count/count)

0.006334841628959276
0.6742081447963801
0.0018099547511312218
0.6624434389140271
0.0018099547511312218
0.6841628959276018
0.0027149321266968325
0.6850678733031674
0.0027149321266968325
0.6678733031674208
0.0036199095022624436
0.6787330316742082
0.0018099547511312218
0.6886877828054299
0.004524886877828055
0.6606334841628959
0.0018099547511312218
0.6624434389140271
0.00904977375565611
0.6687782805429864


In [7]:
np.mean(accuracy)

0.6733031674208145

In [8]:
regret_list[train_len-1]

1615

In [10]:
with open('results/linucb_regret.pkl', 'wb') as f:
    pickle.dump(regret_list, f)
with open('results/linucb_accuracy.pkl', 'wb') as f:
    pickle.dump(accuracy_list, f)